In [2]:
%pylab inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style()

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os, sys, time
import cPickle as pickle
sys.path.append('../EXP/')
import ClevelandMcGill as C
reload(C)

<module 'ClevelandMcGill' from '../EXP/ClevelandMcGill/__init__.pyc'>

In [4]:
NOISE = True

In [5]:
train_target = 60000
val_target = 20000
test_target = 20000

X_train = np.zeros((train_target, 100, 100), dtype=np.float32)
y_train = np.zeros((train_target), dtype=np.float32)
train_counter = 0

X_val = np.zeros((val_target, 100, 100), dtype=np.float32)
y_val = np.zeros((val_target), dtype=np.float32)
val_counter = 0

X_test = np.zeros((test_target, 100, 100), dtype=np.float32)
y_test = np.zeros((test_target), dtype=np.float32)
test_counter = 0

t0 = time.time()

all_counter = 0
while train_counter < train_target or val_counter < val_target or test_counter < test_target:
  
  all_counter += 1
  
  sparse, image, label, parameters = C.Figure1.angle()
  
  if label == 0:
    break
  
  # we need float
  image = image.astype(np.float32)
  
  pot = np.random.choice(3, p=([.6,.2,.2]))

  if pot == 0 and train_counter < train_target:
    # a training candidate
    if label in y_val or label in y_test:
      # no thank you
      continue
      
    # add noise?
    if NOISE:
      image += np.random.uniform(0, 0.05,(100,100))
      
    # safe to add to training
    X_train[train_counter] = image
    y_train[train_counter] = label
    train_counter += 1
    
  elif pot == 1 and val_counter < val_target:
    # a validation candidate
    if label in y_train or label in y_test:
      # no thank you
      continue
      
    # add noise?
    if NOISE:
      image += np.random.uniform(0, 0.05,(100,100))
      
    # safe to add to validation
    X_val[val_counter] = image
    y_val[val_counter] = label
    val_counter += 1
  
  elif pot == 2 and test_counter < test_target:
    # a test candidate
    if label in y_train or label in y_val:
      # no thank you
      continue
      
    # add noise?
    if NOISE:
      image += np.random.uniform(0, 0.05,(100,100))
      
    # safe to add to test
    X_test[test_counter] = image
    y_test[test_counter] = label
    test_counter += 1
  
  
  
print 'Done', time.time()-t0, 'seconds (', all_counter, 'iterations)'

Done 110.705868006 seconds ( 566244 iterations)


In [6]:
X_min = min(X_train.min(), X_val.min(), X_test.min())
X_max = max(X_train.max(), X_val.max(), X_test.max())
y_min = min(y_train.min(), y_val.min(), y_test.min())
y_max = max(y_train.max(), y_val.max(), y_test.max())

# scale in place
X_train -= X_min
X_train /= (X_max - X_min)
X_val -= X_min
X_val /= (X_max - X_min)
X_test -= X_min
X_test /= (X_max - X_min)
y_train -= y_min
y_train /= (y_max - y_min)
y_val -= y_min
y_val /= (y_max - y_min)
y_test -= y_min
y_test /= (y_max - y_min)

# normalize to -.5 .. .5
X_train -= .5
X_val -= .5
X_test -= .5

print 'memory usage', (X_train.nbytes + X_val.nbytes + X_test.nbytes + y_train.nbytes + y_val.nbytes + y_test.nbytes) / 1000000., 'MB'

memory usage 4000.4 MB


In [7]:
from keras import models
from keras import layers
from keras import optimizers
import keras.applications
from keras import backend as K
from keras.utils.np_utils import to_categorical

classifier = models.Sequential()
classifier.add(layers.Convolution2D(20, (5, 5), padding="same", input_shape=(100, 100, 1)))
classifier.add(layers.Activation("relu"))
classifier.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
classifier.add(layers.Dropout(0.2))
classifier.add(layers.Convolution2D(50, (5, 5), padding="same"))
classifier.add(layers.Activation("relu"))
classifier.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
classifier.add(layers.Dropout(0.2))
classifier.add(layers.Flatten())



Using TensorFlow backend.


In [11]:
feature_shape = classifier.output_shape

In [12]:
from keras import models
from keras import layers
from keras import optimizers
import keras.applications
from keras import backend as K
from keras.utils.np_utils import to_categorical

#MLP = models.Sequential()
MLP = classifier
MLP.add(layers.Dense(256, activation='relu', input_dim=feature_shape))
MLP.add(layers.Dropout(0.5))
MLP.add(layers.Dense(1, activation='linear')) # REGRESSION

sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
MLP.compile(loss='mean_squared_error', optimizer=sgd, metrics=['mse', 'mae']) # MSE for regression

In [ ]:
t0 = time.time()
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')]

history = MLP.fit(X_train.reshape(len(X_train), 100, 100, 1), \
                  y_train, \
                  epochs=1000, \
                  batch_size=32, \
                  validation_data=(X_val.reshape(len(X_val), 100, 100, 1), y_val),
                  callbacks=callbacks,
                  verbose=True)
print 'Fitting done', time.time()-t0

Train on 60000 samples, validate on 20000 samples
Epoch 1/1000
60000/60000 [==============================] - 63s 1ms/step - loss: 0.0953 - mean_squared_error: 0.0953 - mean_absolute_error: 0.2680 - val_loss: 0.1078 - val_mean_squared_error: 0.1078 - val_mean_absolute_error: 0.2804
Epoch 2/1000
60000/60000 [==============================] - 61s 1ms/step - loss: 0.0853 - mean_squared_error: 0.0853 - mean_absolute_error: 0.2564 - val_loss: 0.0850 - val_mean_squared_error: 0.0850 - val_mean_absolute_error: 0.2428
Epoch 3/1000
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0715 - mean_squared_error: 0.0715 - mean_absolute_error: 0.2348 - val_loss: 0.0578 - val_mean_squared_error: 0.0578 - val_mean_absolute_error: 0.1959
Epoch 4/1000
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0571 - mean_squared_error: 0.0571 - mean_absolute_error: 0.2083 - val_loss: 0.0450 - val_mean_squared_error: 0.0450 - val_mean_absolute_error: 0.1714
Epoch 5/1000
60000

Epoch 36/1000
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0085 - mean_squared_error: 0.0085 - mean_absolute_error: 0.0736 - val_loss: 0.0019 - val_mean_squared_error: 0.0019 - val_mean_absolute_error: 0.0317
Epoch 37/1000
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0084 - mean_squared_error: 0.0084 - mean_absolute_error: 0.0732 - val_loss: 0.0018 - val_mean_squared_error: 0.0018 - val_mean_absolute_error: 0.0311
Epoch 38/1000
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0084 - mean_squared_error: 0.0084 - mean_absolute_error: 0.0729 - val_loss: 0.0018 - val_mean_squared_error: 0.0018 - val_mean_absolute_error: 0.0318
Epoch 39/1000
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0082 - mean_squared_error: 0.0082 - mean_absolute_error: 0.0725 - val_loss: 0.0018 - val_mean_squared_error: 0.0018 - val_mean_absolute_error: 0.0314
Epoch 40/1000
60000/60000 [==============================] - 62s

60000/60000 [==============================] - 62s 1ms/step - loss: 0.0069 - mean_squared_error: 0.0069 - mean_absolute_error: 0.0664 - val_loss: 0.0015 - val_mean_squared_error: 0.0015 - val_mean_absolute_error: 0.0283
Epoch 72/1000
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0069 - mean_squared_error: 0.0069 - mean_absolute_error: 0.0663 - val_loss: 0.0016 - val_mean_squared_error: 0.0016 - val_mean_absolute_error: 0.0283
Epoch 73/1000
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0068 - mean_squared_error: 0.0068 - mean_absolute_error: 0.0658 - val_loss: 0.0016 - val_mean_squared_error: 0.0016 - val_mean_absolute_error: 0.0282
Epoch 74/1000
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0068 - mean_squared_error: 0.0068 - mean_absolute_error: 0.0659 - val_loss: 0.0016 - val_mean_squared_error: 0.0016 - val_mean_absolute_error: 0.0285
Epoch 75/1000
60000/60000 [==============================] - 62s 1ms/step - lo

In [ ]:
y_pred = MLP.predict(X_test.reshape(len(X_test), 100, 100, 1))

In [ ]:
import sklearn.metrics
np.log2(sklearn.metrics.mean_absolute_error(y_pred*100, y_test*100)+.125)

In [13]:
y_pred

array([[ 0.22938162],
       [ 0.14699593],
       [ 0.6785295 ],
       ..., 
       [ 0.71962476],
       [ 0.86336124],
       [ 0.2867682 ]], dtype=float32)

In [14]:
y_test

array([ 0.25842696,  0.14606741,  0.75280899, ...,  0.75280899,
        0.86516851,  0.29213482], dtype=float32)

In [15]:
sklearn.metrics.mean_absolute_error(y_pred*100, y_test*100)

2.8876698